In [4]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [21]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']

preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']

period = params['periods']
summary_dir = params['summary_dir']

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")
transit_skims_dir = _join(skims_dir, "transit")

annual_transit_factor = params['annual_transit_factor']
annual_auto_factor = params['annual_auto_factor']

filename_extension = params['filename_extension']

In [6]:
perf_measure_columns

['Concept_ID',
 'Metric_ID',
 'Metric_name',
 'Submetric',
 'Description',
 'Population',
 'Period',
 'Geography',
 'Zone_ID',
 'Origin_zone',
 'Dest_zone',
 'Purpose',
 'Mode',
 'Income',
 'Value',
 'Units',
 'Total_Increment']

In [3]:
# Create RAIL Inclusive OD pairs
create_rail_od_pairs(preprocess_dir, transit_skims_dir, period, acc_egg_modes)

Period:  am
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  md
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  pm
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  ev
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR
Period:  ea
Access Egress Mode:  WLK_TRN_WLK
Access Egress Mode:  KNR_TRN_WLK
Access Egress Mode:  PNR_TRN_WLK
Access Egress Mode:  WLK_TRN_PNR
Access Egress Mode:  WLK_TRN_KNR


In [3]:
df_trips = pd.read_parquet(_join(preprocess_dir, 'trip_roster.parquet'))
df_trn = df_trips.loc[df_trips['trip_mode'].isin([6,7,8])]
#df_trn = df_trn.rename(columns={'income_bin' : 'Income'})

In [7]:
%%time

df_temp = []

for period in time_periods:
    print(f'processing - {period}')
    
    df_od_pr = omx.open_file(_join(preprocess_dir, "rail_od_v9_trim_" + period.upper() + ".omx"))
    df_trn_pd = df_trn[df_trn['Period'] == period]
    
    #walk transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'WALK_TRANSIT']
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_WLK')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_wlk = pd.merge(df_trn_acc, df_rail_od, 
                          left_on =['orig_taz', 'dest_taz'], 
                          right_on=['orig', 'dest'], 
                          how ='inner')
    
    # PNR Transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'PNR_TRANSIT']
    df_trn_acc_inbnd = df_trn_acc[df_trn_acc['inbound'] == 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_PNR')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_pnr_inb = pd.merge(df_trn_acc_inbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_acc_outbnd = df_trn_acc[df_trn_acc['inbound'] != 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'PNR_TRN_WLK')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_pnr_outbnd = pd.merge(df_trn_acc_outbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_pnr = pd.concat([df_trn_pnr_inb, df_trn_pnr_outbnd], ignore_index=True)
    
    # KNR Transit
    df_trn_acc = df_trn_pd[df_trn_pd['Mode'] == 'KNR_TRANSIT']
    df_trn_acc_inbnd = df_trn_acc[df_trn_acc['inbound'] == 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'WLK_TRN_KNR')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_knr_inb = pd.merge(df_trn_acc_inbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')


    df_trn_acc_outbnd = df_trn_acc[df_trn_acc['inbound'] != 1] # returning home
    df_rail_od = skim_core_to_df(df_od_pr, 'KNR_TRN_WLK')
    df_rail_od = df_rail_od[df_rail_od['rail_od'] > 0]
    df_trn_knr_outbnd = pd.merge(df_trn_acc_outbnd, df_rail_od, 
                              left_on =['orig_taz', 'dest_taz'], 
                              right_on=['orig', 'dest'], how ='inner')

    df_trn_knr = pd.concat([df_trn_knr_inb, df_trn_knr_outbnd], ignore_index=True)
    
    df_trn_rail = pd.concat([df_trn_wlk, df_trn_pnr, df_trn_knr], ignore_index=True)
    df_temp.append(df_trn_rail)

df_trn_rail = pd.concat(df_temp)

processing - am
processing - md
processing - pm
processing - ev
processing - ea
Wall time: 4min 42s


In [8]:
df_trn_rail['link21_trip_purp'].value_counts()

work        614016
social       93426
othmaint     64672
school       54056
shopping     53901
othdiscr     51646
escort       26677
business       662
Name: link21_trip_purp, dtype: int64

In [23]:
groupby_columns = ['Period', 'Mode', 'link21_trip_purp', 'Income']
summary_column = 'trips'

In [24]:
# Regional Summary

# df_region_period = df_trn_rail.groupby(['Mode', 'Period', 'link21_trip_purp', 'Income'])['trips'].sum().reset_index()

# Call the summarize_all_combinations function
df_region_period = summarize_all_combinations(df_trn_rail, groupby_columns, summary_column)

#df_region_period['Value'] = df_region_period['trips']
df_region_period = df_region_period.rename(columns={'link21_trip_purp': 'Purpose'})
df_region_period = df_region_period[['Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_region_period['Concept_ID'] = concept_id
df_region_period['Metric_ID'] = 'A3.1'
df_region_period['Metric_name'] = 'Linked trips'
df_region_period['Submetric'] = 'A3.1.1'
df_region_period['Description'] = 'Average weekday regional linked trips by time period and purpose'
df_region_period['Population'] = 'Whole Population'
df_region_period['Geography'] = 'Region'
df_region_period['Zone_ID'] = ''
df_region_period['Origin_zone'] = ''
df_region_period['Dest_zone'] = ''
df_region_period['Units'] = 'trips'
df_region_period['Total_Increment'] = ''

In [25]:
#df_tb = df_trn_rail.groupby(['transbay_od', 'Mode', 'Period', 'link21_trip_purp', 'Income'])['trips'].sum().reset_index()

df_tb = df_trn_rail.loc[df_trn_rail['transbay_od'] > 0]
df_tb = summarize_all_combinations(df_tb, groupby_columns, summary_column)

#df_tb['Value'] = df_tb['trips']
df_tb = df_tb.rename(columns={'link21_trip_purp': 'Purpose'})

df_tb = df_tb[['Mode', 'Period', 'Purpose', 'Income' ,'Value']]

df_tb['Concept_ID'] = concept_id
df_tb['Metric_ID'] = 'A3.1'
df_tb['Metric_name'] = 'Linked trips'
df_tb['Submetric'] = 'A3.1.2'
df_tb['Description'] = 'Average weekday linked trips by time period and purpose in transbay region'
df_tb['Population'] = 'Whole Population'
df_tb['Geography'] = 'Transbay region'
df_tb['Origin_zone'] = ''
df_tb['Dest_zone'] = ''
df_tb['Zone_ID'] = ''
df_tb['Units'] = 'trips'
df_tb['Total_Increment'] = ''

In [26]:
df_cnty = df_trn_rail.groupby(['orig_county', 'dest_county', 'Mode', 
                            'Period', 'link21_trip_purp', 'Income'])['trips'].sum().reset_index()

groupby_columns = ['orig_county', 'dest_county', 'Period', 'Mode', 'link21_trip_purp', 'Income']
summary_column = 'trips'

df_cnty = summarize_all_combinations(df_cnty, groupby_columns, summary_column)
#df_cnty['Value'] = df_cnty['trips']
df_cnty = df_cnty.rename(columns={
                                  'link21_trip_purp': 'Purpose', 
                                  'orig_county' : 'Origin_zone',
                                  'dest_county' : 'Dest_zone'})
df_cnty = df_cnty[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_cnty['Concept_ID'] = concept_id
df_cnty['Metric_ID'] = 'A3.1'
df_cnty['Metric_name'] = 'Linked trips'
df_cnty['Submetric'] = 'A3.1.3'
df_cnty['Description'] = 'Average weekday linked trips between origin and destination county by time period and purpose'
df_cnty['Population'] = 'Whole Population'
df_cnty['Geography'] = 'County'
df_cnty['Zone_ID'] = ''
df_cnty['Units'] = 'trips'
df_cnty['Total_Increment'] = ''

In [27]:
df_rdm = df_trn_rail.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Mode', 
                           'Period', 'link21_trip_purp', 'Income'])['trips'].sum().reset_index()

groupby_columns = ['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Mode', 'link21_trip_purp', 'Income']
summary_column = 'trips'

df_rdm = summarize_all_combinations(df_rdm, groupby_columns, summary_column)

#df_rdm['Value'] = df_rdm['trips']
df_rdm = df_rdm.rename(columns={
                                'link21_trip_purp': 'Purpose', 
                                'orig_rdm_zones' : 'Origin_zone',
                                'dest_rdm_zones' : 'Dest_zone'})

df_rdm = df_rdm[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_rdm['Concept_ID'] = concept_id
df_rdm['Metric_ID'] = 'A3.1'
df_rdm['Metric_name'] = 'Linked trips'
df_rdm['Submetric'] = 'A3.1.4'
df_rdm['Description'] = 'Average weekday Linked trips between origin and destination RDM zones by time period and purpose'
df_rdm['Population'] = 'Whole Population'
df_rdm['Geography'] = 'RDM'
df_rdm['Zone_ID'] = ''
df_rdm['Units'] = 'trips'
df_rdm['Total_Increment'] = ''

In [28]:
df_sd = df_trn_rail.groupby(['orig_super_dist', 'dest_super_dist', 'Mode', 
                          'Period', 'link21_trip_purp', 'Income'])['trips'].sum().reset_index()

groupby_columns = ['orig_super_dist', 'dest_super_dist', 'Period', 'Mode', 'link21_trip_purp', 'Income']
summary_column = 'trips'

df_sd = summarize_all_combinations(df_sd, groupby_columns, summary_column)

#df_sd['Value'] = df_sd['trips']

df_sd = df_sd.rename(columns={ 
                              'link21_trip_purp': 'Purpose', 
                              'orig_super_dist' : 'Origin_zone',
                              'dest_super_dist' : 'Dest_zone'})

df_sd = df_sd[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_sd['Concept_ID'] = concept_id
df_sd['Metric_ID'] = 'A3.1'
df_sd['Metric_name'] = 'Linked trips'
df_sd['Submetric'] = 'A3.1.5'
df_sd['Description'] = 'Average weekday linked trips between origin and destination super districts by time period and purpose'
df_sd['Population'] = 'Whole Population'
df_sd['Geography'] = 'Super district'
df_sd['Zone_ID'] = ''
df_sd['Units'] = 'trips'
df_sd['Total_Increment'] = ''

In [29]:
df_trn_rail['pp_trips'] = df_trn_rail['trips'] * df_trn_rail['pp_share']

df_pp = df_trn_rail.groupby(['Mode', 'Period', 'link21_trip_purp', 'Income'])['pp_trips'].sum().reset_index()
#df_pp['Value'] = df_pp['pp_trips']

df_pp = summarize_all_combinations(df_pp, groupby_columns=['Period', 'Mode', 'link21_trip_purp', 'Income'], 
                                   summary_column='pp_trips')

df_pp = df_pp.rename(columns={'link21_trip_purp': 'Purpose'})

df_pp = df_pp[['Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_pp['Concept_ID'] = concept_id
df_pp['Metric_ID'] = 'A3.1'
df_pp['Metric_name'] = 'Linked trips'
df_pp['Submetric'] = 'A3.1.6'
df_pp['Description'] = 'Average weekday Linked trips between by time period and purpose'
df_pp['Population'] = 'Prioirty population'
df_pp['Geography'] = 'Region'
df_pp['Zone_ID'] = ''
df_pp['Origin_zone'] = ''
df_pp['Dest_zone'] = ''
df_pp['Units'] = 'trips'
df_pp['Total_Increment'] = ''

## Write Outputs

In [30]:
all_dfs = [df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]

for dfs in all_dfs:
    metric_name = '_weekday_linked_trips_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    file_name = dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0])
    
combined_df = pd.concat([df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]).reset_index(drop=True)
combined_df.to_csv(_join(summary_dir, 'A3.1' + '_weekday_linked_trips_' + concept_id + '_region' +filename_extension+'.csv'), index=None)

997 A3.1.1 Linked trips
973 A3.1.2 Linked trips
43928 A3.1.3 Linked trips
3688644 A3.1.4 Linked trips
304294 A3.1.5 Linked trips
997 A3.1.6 Linked trips


In [31]:
all_dfs = [df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]

i=7
for dfs in all_dfs:
    
    metric_name = '_annual_linked_trips_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    dfs['Value'] = dfs['Value'] * annual_transit_factor
    dfs['Description'] = dfs['Description'].str.replace('Average weekday','Annual')
    file_name = 'A3.1.' + str(i) #dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0], )
    i=i+1
    
combined_df = pd.concat([df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]).reset_index(drop=True)
combined_df.to_csv(_join(summary_dir, 'A3.1' + '_annual_linked_trips_' + concept_id + '_region' + filename_extension + '.csv'), index=None)

997 A3.1.7 Linked trips
973 A3.1.8 Linked trips
43928 A3.1.9 Linked trips
3688644 A3.1.10 Linked trips
304294 A3.1.11 Linked trips
997 A3.1.12 Linked trips


In [118]:
#TO DO : Priority Population

In [119]:
#with pd.ExcelWriter(os.path.join(summary_outputs, 'concept-BY15.xlsx'), engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
#    df_region_period.to_excel(writer, sheet_name='A3.1.1', startcol=0, index=False)
#    df_tb.to_excel(writer, sheet_name='A3.1.2', startcol=0, index=False)
#    df_cnty.to_excel(writer, sheet_name='A3.1.3', startcol=0, index=False)
#    df_rdm.to_excel(writer, sheet_name='A3.1.4', startcol=0, index=False)
#    df_sd.to_excel(writer, sheet_name='A3.1.5', startcol=0, index=False)

## B1.2 and B1.4 Commuting Trips - Work and Non-work trips

In [32]:
commuting_purposes = params['commuting_purpose_mapping']
commuting_purposes

{'work': 'work',
 'social': 'non-work',
 'othmaint': 'non-work',
 'school': 'school',
 'othdiscr': 'non-work',
 'shopping': 'non-work',
 'escort': 'non-work',
 'business': 'non-work'}

In [33]:
df_trn_rail['wrk_nwrk_purp'] = df_trn_rail['link21_trip_purp'].map(commuting_purposes)

In [34]:
work_trips = df_trn_rail.loc[df_trn_rail['wrk_nwrk_purp'].isin(['work', 'school'])]

In [35]:
groupby_columns = ['Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'trips'

In [37]:
df_region_period = work_trips.groupby(['Mode', 'Period', 'wrk_nwrk_purp', 'Income'])['trips'].sum().reset_index()

df_region_period = summarize_all_combinations(df_region_period, groupby_columns, summary_column)

df_region_period = df_region_period.rename(columns={'wrk_nwrk_purp': 'Purpose'})
df_region_period = df_region_period[['Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_region_period['Concept_ID'] = concept_id
df_region_period['Metric_ID'] = 'B1.2'
df_region_period['Metric_name'] = 'Commute trips on network'
df_region_period['Submetric'] = 'B1.2.1'
df_region_period['Description'] = 'Average weekday linked rail trips for commuting purpose (work and school trip purpose)'
df_region_period['Population'] = 'Whole Population'
df_region_period['Geography'] = 'Region'
df_region_period['Zone_ID'] = ''
df_region_period['Origin_zone'] = ''
df_region_period['Dest_zone'] = ''
df_region_period['Units'] = 'trips'
df_region_period['Total_Increment'] = ''

In [49]:
#df_tb = work_trips.groupby(['transbay_od', 'Mode', 'Period', 'wrk_nwrk_purp', 'Income'])['trips'].sum().reset_index()

df_tb = work_trips.loc[work_trips['transbay_od'] > 0]
df_tb = summarize_all_combinations(df_tb, groupby_columns, summary_column)

df_tb = df_tb.rename(columns={'wrk_nwrk_purp': 'Purpose'})
df_tb = df_tb[['Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_tb['Concept_ID'] = concept_id
df_tb['Metric_ID'] = 'B1.2'
df_tb['Metric_name'] = 'Commute trips on network'
df_tb['Submetric'] = 'B1.2.2'
df_tb['Description'] = 'Average weekday linked rail trips for commuting purpose (work and school trip purpose) in transbay region'
df_tb['Population'] = 'Whole Population'
df_tb['Geography'] = 'Transbay region'
#df_region_period['Zone_ID'] =
df_tb['Zone_ID'] = ''
df_tb['Origin_zone'] = ''
df_tb['Dest_zone'] = ''
df_tb['Units'] = 'trips'
df_tb['Total_Increment'] = ''

In [50]:
groupby_columns = ['orig_county', 'dest_county', 'Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'trips'

#df_cnty = work_trips.groupby(['orig_county', 'dest_county', 'Mode', 
#                            'Period', 'wrk_nwrk_purp', 'Income'])['trips'].sum().reset_index()

#df_cnty['Value'] = df_cnty['trips']
df_cnty = summarize_all_combinations(work_trips, groupby_columns, summary_column)

df_cnty = df_cnty.rename(columns={
                                  'wrk_nwrk_purp': 'Purpose', 
                                  'orig_county' : 'Origin_zone',
                                  'dest_county' : 'Dest_zone'})
df_cnty = df_cnty[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_cnty['Concept_ID'] = concept_id
df_cnty['Metric_ID'] = 'B1.2'
df_cnty['Metric_name'] = 'Commute trips on network'
df_cnty['Submetric'] = 'B1.2.3'
df_cnty['Description'] = 'Average weekday linked rail trips for commuting purpose(work and school trip purpose) in origin and destination county'
df_cnty['Population'] = 'Whole Population'
df_cnty['Geography'] = 'County'
df_cnty['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_cnty['Units'] = 'trips'
df_cnty['Total_Increment'] = ''

In [43]:
groupby_columns = ['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'trips'

#df_rdm = work_trips.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Mode', 
#                           'Period', 'wrk_nwrk_purp', 'Income'])['trips'].sum().reset_index()

#df_rdm['Value'] = df_rdm['trips']

df_rdm = summarize_all_combinations(work_trips, groupby_columns, summary_column)
df_rdm = df_rdm.rename(columns={
                                'wrk_nwrk_purp': 'Purpose', 
                                'orig_rdm_zones' : 'Origin_zone',
                                'dest_rdm_zones' : 'Dest_zone'})

df_rdm = df_rdm[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_rdm['Concept_ID'] = concept_id
df_rdm['Metric_ID'] = 'B1.2'
df_rdm['Metric_name'] = 'Commute trips on network'
df_rdm['Submetric'] = 'B1.2.4'
df_rdm['Description'] = 'Average weekday linked rail trips for commuting purpose(work and school trip purpose) in origin and destination RDM zone'
df_rdm['Population'] = 'Whole Population'
df_rdm['Geography'] = 'RDM'
df_rdm['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_rdm['Units'] = 'trips'
df_rdm['Total_Increment'] = ''

In [44]:
groupby_columns = ['orig_super_dist', 'dest_super_dist', 'Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'trips'

#f_sd = work_trips.groupby(['orig_super_dist', 'dest_super_dist', 'Mode', 
#                          'Period', 'wrk_nwrk_purp', 'Income'])['trips'].sum().reset_index()

df_sd = summarize_all_combinations(work_trips, groupby_columns, summary_column)


#df_sd['Value'] = df_sd['trips']

df_sd = df_sd.rename(columns={ 
                              'wrk_nwrk_purp': 'Purpose', 
                              'orig_super_dist' : 'Origin_zone',
                              'dest_super_dist' : 'Dest_zone'})
df_sd = df_sd[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_sd['Concept_ID'] = concept_id
df_sd['Metric_ID'] = 'B1.2'
df_sd['Metric_name'] = 'Commute trips on network'
df_sd['Submetric'] = 'B1.2.5'
df_sd['Description'] = 'Average weekday linked rail trips for commuting purpose(work and school trip purpose) in origin and destination super district'
df_sd['Population'] = 'Whole Population'
df_sd['Geography'] = 'Super district'
df_sd['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_sd['Units'] = 'trips'
df_sd['Total_Increment'] = ''

In [45]:
work_trips['pp_trips'] = work_trips['trips'] * work_trips['pp_share']

groupby_columns = ['Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'pp_trips'

df_pp = work_trips.groupby(['Mode', 'Period', 'wrk_nwrk_purp', 'Income'])['pp_trips'].sum().reset_index()

df_pp = summarize_all_combinations(df_pp, groupby_columns=['Period', 'Mode', 'wrk_nwrk_purp', 'Income'], 
                                   summary_column='pp_trips')

#df_pp = summarize_all_combinations(df_pp, groupby_columns=['Period', 'Mode', 'link21_trip_purp', 'Income'], 
#                                   summary_column='pp_trips')

df_pp = df_pp.rename(columns={'wrk_nwrk_purp': 'Purpose'})

#df_pp['Value'] = df_pp['pp_trips']
df_pp = df_pp[['Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_pp['Concept_ID'] = concept_id
df_pp['Metric_ID'] = 'B1.2'
df_pp['Metric_name'] = 'Commute trips on network'
df_pp['Submetric'] = 'B1.2.6'
df_pp['Description'] = 'Average weekday linked rail trips for commuting purpose(work and school trip purpose)'
df_pp['Population'] = 'Prioirty population'
df_pp['Geography'] = 'Region'
df_pp['Zone_ID'] = ''
df_pp['Origin_zone'] = ''
df_pp['Dest_zone'] = ''
df_pp['Units'] = 'trips'
df_pp['Total_Increment'] = ''

In [53]:
all_dfs = [df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]

for dfs in all_dfs:
    metric_name = '_weekday_commute_trips_on_network_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    file_name = dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0])
    
combined_df = pd.concat([df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]).reset_index(drop=True)
combined_df.to_csv(_join(summary_dir, 'B1.2' + '_weekday_commute_trips_on_network_'  + concept_id + '_region' +filename_extension + '.csv'), index=None)

359 B1.2.1 Commute trips on network
358 B1.2.2 Commute trips on network
20671 B1.2.3 Commute trips on network
2402292 B1.2.4 Commute trips on network
165434 B1.2.5 Commute trips on network
359 B1.2.6 Commute trips on network


In [54]:
all_dfs = [df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]
i=7

for dfs in all_dfs:
    metric_name = '_annual_commute_trips_on_network_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    dfs['Value'] = dfs['Value'] * annual_transit_factor
    dfs['Description'] = dfs['Description'].str.replace('Average weekday','Annual')
    file_name = 'B1.2.' + str(i) #dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0])
    i=i+1
    
combined_df = pd.concat([df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]).reset_index(drop=True)
combined_df.to_csv(_join(summary_dir, 'B1.2' + '_annual_commute_trips_on_network_' + concept_id + '_region' + filename_extension + '.csv'), index=None)

359 B1.2.7 Commute trips on network
358 B1.2.8 Commute trips on network
20671 B1.2.9 Commute trips on network
2402292 B1.2.10 Commute trips on network
165434 B1.2.11 Commute trips on network
359 B1.2.12 Commute trips on network


## B1.4 

In [56]:
non_work_trips = df_trn_rail.loc[df_trn_rail['wrk_nwrk_purp'] == 'non-work']

In [57]:
groupby_columns = ['Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'trips'

df_region_period = non_work_trips.groupby(['Mode', 'Period', 'wrk_nwrk_purp', 'Income'])['trips'].sum().reset_index()

df_region_period = summarize_all_combinations(df_region_period, groupby_columns, summary_column)

#df_region_period['Value'] = df_region_period['trips']
df_region_period = df_region_period.rename(columns={'wrk_nwrk_purp': 'Purpose'})
df_region_period = df_region_period[['Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_region_period['Concept_ID'] = concept_id
df_region_period['Metric_ID'] = 'B1.4'
df_region_period['Metric_name'] = 'Non-work trips on network'
df_region_period['Submetric'] = 'B1.4.1'
df_region_period['Description'] = 'Average weekday linked rail trips for non-work purpose (excludes work and school trips)'
df_region_period['Population'] = 'Whole Population'
df_region_period['Geography'] = 'Region'
df_region_period['Zone_ID'] = ''
df_region_period['Origin_zone'] = ''
df_region_period['Dest_zone'] = ''
df_region_period['Units'] = 'trips'
df_region_period['Total_Increment'] = ''

In [59]:
groupby_columns = ['Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'trips'

#df_tb = df_tb.groupby(['transbay_od', 'Mode', 'Period', 'wrk_nwrk_purp', 'Income'])['trips'].sum().reset_index()
df_tb = non_work_trips.loc[non_work_trips['transbay_od'] > 0]
df_tb = summarize_all_combinations(df_tb, groupby_columns, summary_column)

df_tb = df_tb.rename(columns={'wrk_nwrk_purp': 'Purpose'})
df_tb = df_tb[['Mode', 'Period', 'Purpose','Income', 'Value']]

df_tb['Concept_ID'] = concept_id
df_tb['Metric_ID'] = 'B1.4'
df_tb['Metric_name'] = 'Non-work trips on network'
df_tb['Submetric'] = 'B1.4.2'
df_tb['Description'] = 'Average weekday linked rail trips for non-work purpose (excludes work and school trips) in transbay region'
df_tb['Population'] = 'Whole Population'
df_tb['Geography'] = 'Transbay region'
df_tb['Zone_ID'] = 1
df_tb['Origin_zone'] = ''
df_tb['Dest_zone'] = ''
df_tb['Units'] = 'trips'
df_tb['Total_Increment'] = ''

In [60]:
groupby_columns = ['orig_county', 'dest_county', 'Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'trips'

#df_cnty = work_trips.groupby(['orig_county', 'dest_county', 'Mode', 
#                            'Period', 'wrk_nwrk_purp', 'Income'])['trips'].sum().reset_index()

#df_cnty['Value'] = df_cnty['trips']
df_cnty = summarize_all_combinations(non_work_trips, groupby_columns, summary_column)
df_cnty = df_cnty.rename(columns={
                                  'wrk_nwrk_purp': 'Purpose', 
                                  'orig_county' : 'Origin_zone',
                                  'dest_county' : 'Dest_zone'})
df_cnty = df_cnty[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_cnty['Concept_ID'] = concept_id
df_cnty['Metric_ID'] = 'B1.4'
df_cnty['Metric_name'] = 'Non-work trips on network'
df_cnty['Submetric'] = 'B1.4.3'
df_cnty['Description'] = 'Average weekday linked rail trips for non-work purpose (excludes work and school trips) in origin and destination counties'
df_cnty['Population'] = 'Whole Population'
df_cnty['Geography'] = 'County'
df_cnty['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_cnty['Units'] = 'trips'
df_cnty['Total_Increment'] = ''

In [61]:
groupby_columns = ['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'trips'

#df_rdm = work_trips.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Mode', 
#                           'Period', 'wrk_nwrk_purp', 'Income'])['trips'].sum().reset_index()

#df_rdm['Value'] = df_rdm['trips']

df_rdm = summarize_all_combinations(non_work_trips, groupby_columns, summary_column)

df_rdm = df_rdm.rename(columns={
                                'wrk_nwrk_purp': 'Purpose', 
                                'orig_rdm_zones' : 'Origin_zone',
                                'dest_rdm_zones' : 'Dest_zone'})

df_rdm = df_rdm[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_rdm['Concept_ID'] = concept_id
df_rdm['Metric_ID'] = 'B1.4'
df_rdm['Metric_name'] = 'Non-work trips on network'
df_rdm['Submetric'] = 'B1.4.4'
df_rdm['Description'] = 'Average weekday linked rail trips for non-work purpose (excludes work and school trips) in origin and destination RDM zones'
df_rdm['Population'] = 'Whole Population'
df_rdm['Geography'] = 'RDM'
df_rdm['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_rdm['Units'] = 'trips'
df_rdm['Total_Increment'] = ''

In [62]:
groupby_columns = ['orig_super_dist', 'dest_super_dist', 'Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'trips'

#f_sd = work_trips.groupby(['orig_super_dist', 'dest_super_dist', 'Mode', 
#                          'Period', 'wrk_nwrk_purp', 'Income'])['trips'].sum().reset_index()

df_sd = summarize_all_combinations(non_work_trips, groupby_columns, summary_column)
df_sd = df_sd.rename(columns={ 
                              'wrk_nwrk_purp': 'Purpose', 
                              'orig_super_dist' : 'Origin_zone',
                              'dest_super_dist' : 'Dest_zone'})
df_sd = df_sd[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_sd['Concept_ID'] = concept_id
df_sd['Metric_ID'] = 'B1.4'
df_sd['Metric_name'] = 'Non-work trips on network'
df_sd['Submetric'] = 'B1.4.5'
df_sd['Description'] = 'Average weekday linked rail trips for non-work purpose (excludes work and school trips) in orgin and destination super district'
df_sd['Population'] = 'Whole Population'
df_sd['Geography'] = 'Super district'
df_sd['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_sd['Units'] = 'trips'
df_sd['Total_Increment'] = ''

In [63]:
work_trips['pp_trips'] = work_trips['trips'] * work_trips['pp_share']

groupby_columns = ['Period', 'Mode', 'wrk_nwrk_purp', 'Income']
summary_column = 'pp_trips'

df_pp = work_trips.groupby(['Mode', 'Period', 'wrk_nwrk_purp', 'Income'])['pp_trips'].sum().reset_index()

df_pp = summarize_all_combinations(df_pp, groupby_columns=['Period', 'Mode', 'wrk_nwrk_purp', 'Income'], 
                                   summary_column='pp_trips')

df_pp = df_pp.rename(columns={ 'wrk_nwrk_purp': 'Purpose'})

df_pp = df_pp[['Mode', 'Period', 'Purpose', 'Income', 'Value']]

df_pp['Concept_ID'] = concept_id
df_pp['Metric_ID'] = 'B1.4'
df_pp['Metric_name'] = 'Non-work trips on network'
df_pp['Submetric'] = 'B1.4.6'
df_pp['Description'] = 'Average weekday linked rail trips for non-work purpose (excludes work and school trips)'
df_pp['Population'] = 'Prioirty population'
df_pp['Geography'] = 'Region'
df_pp['Zone_ID'] = ''
df_pp['Origin_zone'] = ''
df_pp['Dest_zone'] = ''
df_pp['Units'] = 'trips'
df_pp['Total_Increment'] = ''

In [64]:
all_dfs = [df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]

for dfs in all_dfs:
    metric_name = '_weekday_non_work_trips_on_network_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    file_name = dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0])
    
combined_df = pd.concat([df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]).reset_index(drop=True)
combined_df.to_csv(_join(summary_dir, 'B1.4' + '_weekday_non_work_trips_on_network_' + concept_id + '_region' +filename_extension + '.csv'), index=None)

240 B1.4.1 Non-work trips on network
240 B1.4.2 Non-work trips on network
12176 B1.4.3 Non-work trips on network
1175776 B1.4.4 Non-work trips on network
92184 B1.4.5 Non-work trips on network
359 B1.4.6 Non-work trips on network


In [65]:
all_dfs = [df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]

i=7
for dfs in all_dfs:   
    metric_name = '_annual_non_work_trips_on_network_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    dfs['Value'] = dfs['Value'] * annual_transit_factor
    dfs['Description'] = dfs['Description'].str.replace('Average weekday','Annual')
    file_name = 'B1.4.' + str(i) #dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0])
    i=i+1
    
combined_df = pd.concat([df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]).reset_index(drop=True)
combined_df.to_csv(_join(summary_dir, 'B1.4' + '_annual_non_work_trips_on_network_' + concept_id + '_region' +filename_extension + '.csv'), index=None)

240 B1.4.7 Non-work trips on network
240 B1.4.8 Non-work trips on network
12176 B1.4.9 Non-work trips on network
1175776 B1.4.10 Non-work trips on network
92184 B1.4.11 Non-work trips on network
359 B1.4.12 Non-work trips on network


## C2.2

In [66]:
df_business = df_trn_rail.loc[df_trn_rail['link21_trip_purp'] == 'business']

In [67]:
groupby_columns = ['Period', 'Mode', 'Income']
summary_column = 'trips'

df_region_period = summarize_all_combinations(df_business, groupby_columns, summary_column)

df_region_period = df_region_period.rename(columns={'link21_trip_purp': 'Purpose'})
df_region_period = df_region_period[['Mode', 'Period', 'Income', 'Value']]

df_region_period['Concept_ID'] = concept_id
df_region_period['Metric_ID'] = 'C2.2'
df_region_period['Metric_name'] = 'Business to Business trips'
df_region_period['Submetric'] = 'C2.2.1'
df_region_period['Description'] = 'Average weekday linked rail trips for business purpose'
df_region_period['Population'] = 'Whole Population'
df_region_period['Geography'] = 'Region'
df_region_period['Zone_ID'] = ''
df_region_period['Purpose'] = 'business'
df_region_period['Origin_zone'] = ''
df_region_period['Dest_zone'] = ''
df_region_period['Units'] = 'trips'
df_region_period['Total_Increment'] = ''

In [70]:
df_tb = df_business.loc[df_business['transbay_od'] > 0]
df_tb = summarize_all_combinations(df_tb, groupby_columns, summary_column)

df_tb = df_tb.rename(columns={'link21_trip_purp': 'Purpose'})
df_tb = df_tb[['Mode', 'Period', 'Income', 'Value']]

df_tb['Concept_ID'] = concept_id
df_tb['Metric_ID'] = 'C2.2'
df_tb['Metric_name'] = 'Business to Business trips'
df_tb['Submetric'] = 'C2.2.2'
df_tb['Description'] = 'Average weekday linked rail trips for business purpose in transbay region'
df_tb['Population'] = 'Whole Population'
df_tb['Geography'] = 'Transbay region'
df_tb['Zone_ID'] = ''
df_tb['Purpose'] = 'business'
df_tb['Origin_zone'] = ''
df_tb['Dest_zone'] = ''
df_tb['Units'] = 'trips'
df_tb['Total_Increment'] = ''

In [74]:
df_cnty = df_business.groupby(['orig_county', 'dest_county', 'Mode', 
                            'Period', 'Income'])['trips'].sum().reset_index()

groupby_columns = ['orig_county', 'dest_county', 'Period', 'Mode', 'Income']
summary_column = 'trips'

df_cnty = summarize_all_combinations(df_cnty, groupby_columns, summary_column)

df_cnty = df_cnty.rename(columns={
                                  'orig_county' : 'Origin_zone',
                                  'dest_county' : 'Dest_zone'})

df_cnty = df_cnty[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Income', 'Value']]

df_cnty['Concept_ID'] = concept_id
df_cnty['Metric_ID'] = 'C2.2'
df_cnty['Metric_name'] = 'Business to Business trips'
df_cnty['Submetric'] = 'C2.2.3'
df_cnty['Description'] = 'Average weekday linked rail trips for business purpose in origin and destination county'
df_cnty['Population'] = 'Whole Population'
df_cnty['Geography'] = 'County'
df_cnty['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_cnty['Purpose'] = 'business'
df_cnty['Units'] = 'trips'
df_cnty['Total_Increment'] = ''

In [76]:
df_rdm = df_business.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Mode', 
                              'Period', 'Income'])['trips'].sum().reset_index()

groupby_columns = ['orig_rdm_zones', 'dest_rdm_zones', 'Period', 'Mode', 'Income']
summary_column = 'trips'

df_rdm = summarize_all_combinations(df_rdm, groupby_columns, summary_column)

df_rdm = df_rdm.rename(columns={ 
                                'orig_rdm_zones' : 'Origin_zone',
                                'dest_rdm_zones' : 'Dest_zone'})

df_rdm = df_rdm[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Income', 'Value']]

df_rdm['Concept_ID'] = concept_id
df_rdm['Metric_ID'] = 'C2.2'
df_rdm['Metric_name'] = 'Business to Business trips'
df_rdm['Submetric'] = 'C2.2.4'
df_rdm['Description'] = 'Average weekday linked rail trips for business purpose in origin and destination RDM zones'
df_rdm['Population'] = 'Whole Population'
df_rdm['Geography'] = 'RDM'
df_rdm['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_rdm['Purpose'] = 'business'
df_rdm['Units'] = 'trips'
df_rdm['Total_Increment'] = ''

In [78]:
df_sd = df_business.groupby(['orig_super_dist', 'dest_super_dist', 'Mode', 
                             'Period', 'Income'])['trips'].sum().reset_index()

groupby_columns = ['orig_super_dist', 'dest_super_dist', 'Period', 'Mode', 'Income']
summary_column = 'trips'

df_sd = summarize_all_combinations(df_sd, groupby_columns, summary_column)

df_sd = df_sd.rename(columns={ 
                              'orig_super_dist' : 'Origin_zone',
                              'dest_super_dist' : 'Dest_zone'})

df_sd = df_sd[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Income', 'Value']]

df_sd['Concept_ID'] = concept_id
df_sd['Metric_ID'] = 'C2.2'
df_sd['Metric_name'] = 'Business to Business trips'
df_sd['Submetric'] = 'C2.2.5'
df_sd['Description'] = 'Average weekday linked rail trips for business purpose in origin and destination super districts'
df_sd['Population'] = 'Whole Population'
df_sd['Geography'] = 'Super district'
df_sd['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_sd['Purpose'] = 'business'
df_sd['Units'] = 'trips'
df_sd['Total_Increment'] = ''

In [79]:
df_business['pp_trips'] = df_business['trips'] * df_business['pp_share']

df_pp = df_business.groupby(['Mode', 'Period', 'Income'])['pp_trips'].sum().reset_index()

df_pp = summarize_all_combinations(df_pp, groupby_columns=['Period', 'Mode','Income'], 
                                   summary_column='pp_trips')

df_pp = df_pp[['Mode', 'Period', 'Income', 'Value']]

df_pp['Concept_ID'] = concept_id
df_pp['Metric_ID'] = 'C2.2'
df_pp['Metric_name'] = 'Business to Business trips'
df_pp['Submetric'] = 'C2.2.6'
df_pp['Description'] = 'Average weekday linked rail trips for business purpose'
df_pp['Population'] = 'Prioirty population'
df_pp['Geography'] = 'Region'
df_pp['Zone_ID'] = ''
df_pp['Origin_zone'] = ''
df_pp['Dest_zone'] = ''
df_pp['Purpose'] = ''
df_pp['Units'] = 'trips'
df_pp['Total_Increment'] = ''

In [81]:
all_dfs = [df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]

for dfs in all_dfs:
    metric_name = '_weekday_business_to_business_trips_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    file_name = dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0])
    
combined_df = pd.concat([df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]).reset_index(drop=True)
combined_df.to_csv(_join(summary_dir, 'C2.2' + '_weekday_business_to_business_trips_' + concept_id + '_region' +filename_extension+'.csv'), index=None)

44 C2.2.1 Business to Business trips
34 C2.2.2 Business to Business trips
770 C2.2.3 Business to Business trips
6936 C2.2.4 Business to Business trips
2080 C2.2.5 Business to Business trips
44 C2.2.6 Business to Business trips


In [82]:
all_dfs = [df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]
i=7
for dfs in all_dfs:

    metric_name = '_annual_business_to_business_trips_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    dfs['Value'] = dfs['Value'] * annual_transit_factor
    dfs['Description'] = dfs['Description'].str.replace('Average weekday','Annual')
    file_name = 'C2.2.' + str(i) #dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0])
    i=i+1
    
combined_df = pd.concat([df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]).reset_index(drop=True)
combined_df.to_csv(_join(summary_dir, 'C2.2'+'_annual_business_to_business_trips_' + concept_id + '_region' +filename_extension+'.csv'), index=None)

44 C2.2.7 Business to Business trips
34 C2.2.8 Business to Business trips
770 C2.2.9 Business to Business trips
6936 C2.2.10 Business to Business trips
2080 C2.2.11 Business to Business trips
44 C2.2.12 Business to Business trips
